In [25]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import concat
from operator import add

In [2]:
sc = SparkContext(appName="Assignment2_E1")


22/05/19 17:59:37 WARN Utils: Your hostname, Luiss-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.127 instead (on interface en0)
22/05/19 17:59:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 17:59:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
